# A LSTM-CRF Tutorial 
This tutorial aims to introduce an sequence labeling model based on a LSTM-CRF architecture.

## Introduction

Named-entity recognition (NER) is a sub task within information extraction, that seeks to recognize and classify named entity mentions in text into pre-defined semantic categories((e.g., location, organization, geo-political entity, person). For example, given a sentence "Cook bought two hundred shares of Apple Inc. in 2003.", the name entities can be highlighted as:

$[Cook]_{Person} \ bought \ two \ hundred \ shares \ of \ [Apple\  Inc.]_{Organization} \ in  \ [2003]_{Time}.$

In this example, a one-token person name, a two-token company name and a temporal expression have been located and classified. 

NER has a wide range of applications in the industry. It can be used in recognizing relevant entities in customer complaints, which will be classified accordingly and forwarded to the appropriate department responsible for the identified product. It can also be used in search engine algorithms, to extract entities and compare them with the tags associated with the website articles for a quick and efficient search.

NER problem can be viewed as a sequence labeling problem, a simple class of structural prediction problems. The goal is to assign one discrete label to each member of a sequence of input tokens. We may have hearde of Hidden Markov Model (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Fields (CRF), these models can be applied on sequence labeling tasks.

In this tutorial, we will implement a sequence labeling system using CRF, then evaluate it on a NER task. 

## Dataset

The [MIT Movie Corpus]((https://groups.csail.mit.edu/sls/downloads) is a semantically tagged training and test corpus in BIO(short for beginning, inside, outside) format. The original English corpus has been split into the training, development and test sets. You can find them under the "data" folder.

The labeled data has the format of one token per line with label and token separated by tab, and sentences are separated by empty lines. There are three general formats of labels:
- O: the corresponding token is outside of any name entity.
- B-(*category*): the corresponding token is the beginning of a name entity that belongs to the category in the parentheses.
- I-(*category*): the corresponding token is inside a name entity that belongs to the category in the parentheses.

Let us look at a sample sentence in the training set:

**O** did

**B-DIRECTOR** sofia

**I-DIRECTOR** coppola

**O** direct

**O** any

**B-GENRE** adventure

**O** films

In this sample, there are two name entities "sofia coppola" and "adventure", which belong to "DIRECTOR" category and "GENRE" category respectively. The labels "B-DIRECTOR", "I-DIRECTOR" indicate the span of the entity "sofia coppola" whose category is "DIRECTOR".


## Load data

### Task 1
First of all, let us load the sentences and labels from local files. The data has been preprocessed, **don't make any changes to the labels and tokens.**
- Write a function to read the sentences and labels given the path of a text file under the "data" folder.
- Load training, development and test datasets from the text files.

In [ ]:
#Write your code here

In [1]:
from utils import *
import numpy as np
import itertools
from collections import Counter
train_path = "data/ner_train.txt"
dev_path = "data/ner_dev.txt"
test_path = "data/ner_test.txt"
train_sent_tags = load_sentences(train_path)
dev_sent_tags = load_sentences(dev_path)
test_sent_tags = load_sentences(test_path)

In [2]:
train_word_tags = [extract_word_tag(sent_tag) for sent_tag in train_sent_tags]
dev_word_tags = [extract_word_tag(sent_tag) for sent_tag in dev_sent_tags]
test_word_tags = [extract_word_tag(sent_tag) for sent_tag in test_sent_tags]

In [3]:
train_sent_tags, train_sent_words = list(zip(*train_word_tags))
dev_sent_tags, dev_sent_words = list(zip(*dev_word_tags))
test_sent_tags, test_sent_words = list(zip(*test_word_tags))

The tokens and labels must be converted to corresponding indexes for subsequent tasks. The vocabulary and pre-trained embeddings have been provided, run the code below to obtain them. We'll use them later.

In [4]:
import pickle
import numpy as np
path_dict = 'data/vocab/dict.pkl'
path_emb = 'data/vocab/local_emb.pkl'
with open(path_dict, 'rb') as f:
    word2id, id2word, vocab = pickle.load(f)
with open(path_emb, 'rb') as f:
    local_glove_emb = pickle.load(f)
print('Local embeddings size:', local_glove_emb.shape)
print('Vocabulary size', len(vocab))

Local embeddings size: (7482, 100)
Vocabulary size 7482


There are two dictionaries "word2id", "id2word". "word2id" can map a word to its corresponding index in the embeddings, and "id2word" does the opposite. 

In [5]:
print('The index of "the": ', word2id['the'])

The index of "the":  28


There are also dictionaries for labels.

In [6]:
with open('data/labels/dict.pkl', 'rb') as f:
    label2id, id2label, label_vocab = pickle.load(f)
print('Number of unique labels:', len(label2id))
print('The index of label "B-DIRECTOR":', label2id['B-DIRECTOR'])

Number of unique labels: 28
The index of label "B-DIRECTOR": 8


## CRF

CRFs are a type of discriminative undirected probabilistic graphical model. They are used to encode known relationships between observations and construct consistent interpretations and are often used for labeling or parsing of sequential data. CRFs can model complete sequences:

$$p(\mathbf{y} \mid \mathbf{x}) = \frac {exp(f(\mathbf{x}, \mathbf{y}) \cdot  \theta)}{\sum_{\mathbf{y}^{\prime}}exp(f(\mathbf{x}, \mathbf{y}^{\prime}) \cdot \theta)} \ \ \ \ (1)$$

Where $\mathbf{x}$ is a complete token sequence(i.e., a sentence), $\mathbf{y}$ is a complete label sequence, $f(\mathbf{x}, \mathbf{y})$ refers to feature functions and $\theta$ refers to the model parameters. 

To compute the probability $p(\mathbf{y} \mid \mathbf{x})$, we need to figure out: 
- Feature functions. It is very flexible to define feature functions $f(x, y)$, which can be discrete feature functions as discussed in class as well as continuous ones like neural networks. **In this project, we'll use a bi-directional LSTM to extract features from the token sequence.**
- Calculation of the denominator. There are exponentially many possible $y^{\prime}$ here, it is not possible to enumerate all of them. For example, given 5 unique labels, if a sentence has 10 tokens, the number of possible label combinations will be $5^{10}$. **In this project, we'll use the forward algorithm to calculate the denominator efficiently.**


### Task 2

The bidirectional LSTM (biLSTM) class has been provided to extract features of sentences. The input is two Pytorch tensors, one is a batch of token index sequences and the other is the lengths of sequences in the batch.

In [7]:
import torch
import numpy as np
from torch import nn
from lstm_encoder import LSTMEncoder
torch.manual_seed(111)

#Definition of example hyper-parameters
example_vocab_size = 100#example vocabulary size
example_emb_dim = 32#example word embedding dimension
example_hidden_dim = 32#example hidden dimension of LSTM
example_batch_size = 3#example batch size
example_max_seq_len = 10#example max length of sequences
example_label_size = 5#number of unique labels
#Example token sequences
example_sentences = torch.empty(example_batch_size, 
                                example_max_seq_len).random_(example_vocab_size).long()
example_seq_lens = torch.LongTensor([10, 7, 9])#example length of each sequence in the batch
example_labels = torch.empty(example_batch_size, 
                                example_max_seq_len).random_(example_label_size-2).long()

#Create a biLSTM object
lstm = LSTMEncoder(example_vocab_size, example_emb_dim, example_hidden_dim)
sents_lstm_outputs = lstm(example_sentences, example_seq_lens)
print(sents_lstm_outputs.size())

torch.Size([3, 10, 32])


In [8]:
#Convert the lstm outputs to label space for CRF
torch.manual_seed(111)
hidden2label_spcae = nn.Linear(example_hidden_dim, example_label_size)
example_sents_feats = hidden2label_spcae(sents_lstm_outputs)

In [14]:
example_sents_feats.size()

torch.Size([3, 10, 5])

Using the dictionaries that we obtained in Task 1, let us do such work to extract features of sentences:
-  Write a function that can sample a batch of labeled sentences randomly from the training set, or a batch of labeled sentences sequentially from the development (test) set.
- This function returns three tensors: a tensor of token index sequences, a tensor of label index sequences and a tensor of sequence lengths. Pad the token, label sequences in a batch to the same length as we did in homework 1.
- Use this function to generate a batch of data from the training set, feed the data to the biLSTM encoder.

In [ ]:
#Write your code here

In [34]:
from torch.nn.utils.rnn import pad_sequence
def sent2ids(text):
    return [word2id[w] for w in text]

def labels2ids(labels):
    return [label2id[w] for w in labels]


def data_generator(sents, labels, batch_size=32, is_training=True, index=0):
    if is_training:
        select_indices = np.random.choice(len(sents), batch_size, replace=False)
    else:
        start = index
        end = min(start + batch_size, len(sents)) 
        select_indices = list(range(start, end))
    #select_indices = list(range(batch_size))
    batch_sents = np.array(sents)[select_indices]
    batch_labels = np.array(labels)[select_indices]
    
    batch_sents = list(map(sent2ids, batch_sents))
    batch_labels = list(map(labels2ids, batch_labels))
    
    seq_lens = [len(s) for s in batch_sents]
    seq_lens = torch.LongTensor(seq_lens)
    max_len = max(seq_lens)
    
    batch_sents = [torch.LongTensor(s) for s in batch_sents]
    
    
    batch_sents = pad_sequence(batch_sents, batch_first=True)
    
    if not is_training:
        return batch_sents, batch_labels, seq_lens, end
    batch_labels = [torch.LongTensor(s) for s in batch_labels]
    batch_labels = pad_sequence(batch_labels, batch_first=True)
    
    return batch_sents, batch_labels, seq_lens

In [35]:
batch_sents, batch_label, seq_lens = data_generator(train_sent_words, train_sent_tags, batch_size=32)

### Task 3
Let us look at the numerator of Formula 1, we can use score to represent the output of feature functions:

$$score(\mathbf{x}, \mathbf{y})=f(\mathbf{x}, \mathbf{y}) \cdot  \theta \ \ \ \ (2)$$

![crf](imgs/crf2.png)
<center>Figure 1

We'll focus on a linear-chain CRF model that only considers the transitions of two consecutive labels. For any given label sequence, we can connect them one by one and make a path from the start to the end. Take Figure 1 for example, assume the label sequence is "A V P D N", the path is shown as red bold arrows. There's a score for each path in CRF, a good path has a relatively large score. **Note that each path starts from a START label and ends with a STOP label.**

In a linear-chain CRF, the score can be computed as a sum of log potentials:

$$score(\mathbf{x}, \mathbf{y}) = \sum_{j=1}^{n} log\phi(\mathbf{x}, y_{j-1}, y_j, j) + A_{y_{n}, y_{n+1}} \ \ \ \ (3)$$

Where $y_{j-1}$, $y_j$ are the labels at position $j-1$ and $j$ respectively, $n$ is the sequence length, $A \in R^{l \times l}$ is the transition matrix, $l$ is the unique label number, and
$y_{0}=START$ and $y_{n+1} = STOP$.



Similar to HMM, a potential can be denoted as the product of **an emission potential** and **a transition potential**, let us write them in the log form:

$$log\phi(\mathbf{x}, y_{j-1}, y_j, j) = log \phi_{emit}(x_j, y_j) +  log \phi_{trans}(y_{j-1}, y_j) \ \ \ \ (4)$$

$$log \phi_{emit}(y_j, x_j) = (W_xh_j)_{y_j} \ \ \ \ (5)$$

$$log \phi_{trans}(y_{j-1}, y_j) = A_{y_{j-1}, y_j} \ \ \ \ (6) $$

Where $h_j \in R^d$ is the biLSTM output of the token sequence at position $j$, $W_x \in R^{l\times d}$ refers to parameters that map biLSTM outputs to label space, $d$ is the hidden dimension of biLSTM.

With Formula (4), (5) and (6), we can obtain:

$$log\phi(\mathbf{x}, y_{j-1}, y_j, j) = (W_xh_j)_{y_j} + A_{y_{j-1}, y_j} \ \ \ \ (7)$$

**The log emission potentials** can be obtained from the biLSTM output, and **the log transmission potentials can be obtained from the transition matrix.** Therefore, we can compute each potential based on the features extracted by biLSTM and the transition matrix. The transition matrix is viewed as parameters that will be randomly initialized and learned on the training set.

In [21]:
class LinearChainCRF(nn.Module):
    def __init__(self, label_size):
        super(LinearChainCRF, self).__init__()
        
        self.label_size = label_size
        self.start_label = label_size - 1
        self.stop_label = label_size - 2
        self.transitions = nn.Parameter(
            torch.randn(self.label_size, self.label_size))
        self.transitions.data[:, self.start_label] = -10000
        self.transitions.data[self.stop_label] = -10000
     
    def forward_alg(self, features):
        '''
        Forward algorithm
        Arg:
            features: features of a sentence, sent_len*label_size, tensor
        Return:
            alpha: the log sum of 
        '''
        #Complete the code
        init_alphas = torch.full((1, self.label_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0, self.start_label] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in features:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.label_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.label_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[:, next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[:, self.stop_label]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def sentence_scorer(self, features, labels):
        '''
        Score a sentence given its label sequence
        Args:
            features: features of a sentence, sent_len*label_size, tensor
            labels: a sequence of labels, sent_len, tensor
        Return:
            score: the score of the labeled sentence, a scalar
        '''
        #Complete the code
        score = torch.zeros(1)
        labels = torch.cat([torch.tensor([self.start_label], dtype=torch.long), labels])
        for i, feat in enumerate(features):
            score = score + \
                self.transitions[labels[i], labels[i + 1]] + feat[labels[i + 1]]
        score = score + self.transitions[labels[-1], self.stop_label]
        return score
    
    def viterbi_decoder(self, features):
        '''
        Viterbi decoder
        Arg:
            features: features of a sentence, sent_len*label_size, tensor
        Return:
            best_path_score: best path score, a scalar
            best_path: best path, a list
        '''
        #Complete the code
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.label_size), -10000.)
        init_vvars[0][self.start_label] = 0
        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in features:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.label_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[:, next_tag].view(1, -1)
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[:, self.stop_label].view(1, -1)
        best_tag_id = argmax(terminal_var)
        best_path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.start_label  # Sanity check
        best_path.reverse()
        return best_path_score, best_path
    

def argmax(x):
    # return the argmax as a python int
    _, idx = torch.max(x, 1)
    return idx.item()

    
def log_sum_exp(x):
    #return log sum exp of a tensor
    m = torch.max(x, -1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(-1)), -1))

In [23]:
torch.manual_seed(111)
model = LinearChainCRF(example_label_size)
score = model.sentence_scorer(example_sents_feats[0], example_labels[0])
alpha = model.forward_alg(example_sents_feats[0])
path_score, best_path = model.viterbi_decoder(example_sents_feats[0])
print(score, alpha)
print(path_score, best_path)

tensor([-2.7266], grad_fn=<AddBackward0>) tensor([11.1678], grad_fn=<AddBackward0>)
tensor(6.1943, grad_fn=<SelectBackward>) [1, 2, 1, 2, 1, 2, 1, 2, 1, 2]


In [24]:
torch.manual_seed(111)
model2 = LinearChainCRF2(example_label_size)
model2.transitions.data = model.transitions.data.transpose(1, 0)
score = model2._score_sentence(example_sents_feats[0], example_labels[0])
alpha = model2._forward_alg(example_sents_feats[0])
path_score, best_path = model2._viterbi_decode(example_sents_feats[0])
print(score, alpha)
print(path_score, best_path)

tensor([-2.7266], grad_fn=<AddBackward0>) tensor([11.1678], grad_fn=<AddBackward0>)
tensor(6.1943, grad_fn=<SelectBackward>) [1, 2, 1, 2, 1, 2, 1, 2, 1, 2]


A linear-chain CRF class has been created, let us complete such work:
- Complete the function "sentence_scorer" that can compute the score given features of a sentence and a label sequence.
- Run your function on "example_sents_feats" and "example_labels".

You can use pre-defined functions "argmax" and "log_sum_exp".

### Task 4

Using Formula (3) we can compute the score given a sentence features and a label sequence. To obtain the probability of a sequence, we still need to compute the denominator (the sum of all possible exponential values of scores). Let us use log probability instead of probability for convenience:

$$logp(\mathbf{y} \mid \mathbf{x}) =  score(\mathbf{x}, \mathbf{y}) - log(\sum_{\mathbf{y}^{\prime}}exp(score(\mathbf{x}, \mathbf{y}^{\prime})))  \ \ \ \ (8)$$

We'll use the forward algorithm to calculate the log denominator efficiently.
- Complete the function "forward_alg" in the class "LinearChainCRF". Consider the START and the STOP label in your algorithm.
- Run your function on "example_sents_feats".

In [23]:
alpha = model.forward_alg(example_sents_feats[0])
alpha

tensor([11.1678], grad_fn=<AddBackward0>)

### Task 5

Now we have functions to compute both the score of a label sequence and the log denominator, the log probability can be obtained using Formula (9):
- Compute the log probability of the sequence "example_labels" given the sentence features "example_sents_feats".

In [106]:
#Write your code here

In [24]:
score - alpha

tensor([-13.8944], grad_fn=<SubBackward0>)

## Learn parameters

Up to now, we haven't updated any parameters. A good CRF model can learn from the training set and give high probabilities to the gold label sequences by updating the parameters. The loss can be denoted as the negative sum of log probabilities:

$$ -\sum_i^m logp(\mathbf{y_i} \mid \mathbf{x_i}) \ \ \ \ (9)$$

Where $m$ is the number of sentences. The higher the probabilities of gold label sequences, the smaller the loss. Fortunately, we don't need to compute the gradients of parameters as Pytorch will do that automatically.

### Task 6
A "SequenceLabeling" class has been created below to train a CRF model, let us do such work:
- Complete the function "forward"  that compute the loss of a batch of training data.
- Complete the function "load_pretrained_emb" that can load pre-trained Glove embeddings to biLSTM object.

In [25]:
class SequenceLabeling(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, path):
        super(SequenceLabeling, self).__init__()
        self.crf = LinearChainCRF(label_size)
        self.bilstm = LSTMEncoder(vocab_size, embedding_dim, hidden_dim)
        self.hidden2label_space = nn.Linear(hidden_dim, label_size)
        self.load_pretrained_emb(path)
        
        
    def forward(self, sentences, seq_lens, sent_tags):
        '''
        Compute the loss
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
            sent_tags: batch_size*word_num, long tensor
        Return:
            loss: the average loss of a batch
        '''
        hiddens = self.bilstm(sentences, seq_lens)
        #batch_size*word_num*tag_size
        batch_feats = self.hidden2label_space(hiddens)
        neg_likelihoods = []
        for i, feats in enumerate(batch_feats):
            length = seq_lens[i]
            tags = sent_tags[i]
            gold_score = self.crf.sentence_scorer(feats[:length], tags[:length])
            forward_score = self.crf.forward_alg(feats[:length])
            neg_likelihoods.append(forward_score-gold_score)
        loss = torch.stack(neg_likelihoods).mean()
        return loss

        return loss
    
    def decoder(self, sentences, seq_lens):
        '''
        Viterbi decoder
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
        Return:
            pred_path: batch_size, list
        '''
        hiddens = self.bilstm(sentences, seq_lens)
        #batch_size*word_num*tag_size
        batch_feats = self.hidden2label_space(hiddens)
        preds = []
        for i, feats in enumerate(batch_feats):
            length = seq_lens[i]
            _, pred = self.crf.viterbi_decoder(feats[:length])
            preds.append(pred)
        return preds

        return pred_path
    
    def load_pretrained_emb(self, path):
        '''
        Load pre-trained word embeddings from the path
        Arg:
            path: the binary file of local Glove embeddings
        '''
        with open(path, 'rb') as f:
            vectors = pickle.load(f)
            print("Loaded from {} with shape {}".format(path, vectors.shape))
            assert vectors.shape == self.bilstm.word_embeds.weight.size()
            self.bilstm.word_embeds.weight.data.copy_(torch.from_numpy(vectors))
            self.bilstm.word_embeds.weight.requires_grad = False
            print('embeddings loaded')

        

## Evaluate the performance

Similar to constituency-parsing, the evaluation of a NER task is to match the predicted sequence and the gold sequence. We need to generate an optimal label sequence given a sentence. It is impossible to enumerate all of the paths (label sequences) and choose the one with the largest score.

### Task 7
We can use the Viterbi algorithm in choosing the optimal label sequence.
- Complete the function "viterbi_decoder" in the class "LinearChainCRF".
- Complete the function "decoder" in the class "SequenceLabeling".
- Write a function that converts the predicted label index sequences to label sequences. For example, [0, 0, 3, 4, 0, 0] -> ['O', 'O', 'B-ACTOR', 'I-ACTOR', 'O', 'O'].

In [16]:
#Write your code here

In [26]:
from torch import optim
model = SequenceLabeling(len(vocab), 100, 64, len(label2id), 'data/vocab/local_emb.pkl')
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

Loaded from data/vocab/local_emb.pkl with shape (7482, 100)
embeddings loaded


In [28]:
from time import time
epoch = 3
loop_num = int(len(train_sent_words)/32)
for i in range(epoch):
    start = time()
    model.train()
    for j in range(loop_num):
        optimizer.zero_grad()
        batch_sents, batch_tags, seq_lens = data_generator(train_sent_words,train_sent_tags)
        loss = model(batch_sents, seq_lens, batch_tags)
        loss.backward()
        optimizer.step()
        if j % 20 == 0:
            print('Loss:', loss.item())

Loss: 32.355308532714844
Loss: 31.648584365844727
Loss: 18.129901885986328
Loss: 19.353574752807617
Loss: 15.430071830749512
Loss: 16.556926727294922
Loss: 14.95309829711914
Loss: 15.039655685424805
Loss: 18.279136657714844
Loss: 12.75913143157959
Loss: 11.560237884521484
Loss: 9.767963409423828
Loss: 10.669052124023438
Loss: 12.229289054870605
Loss: 8.61056900024414
Loss: 7.509668350219727
Loss: 10.254934310913086
Loss: 8.366771697998047
Loss: 7.92102575302124
Loss: 6.970520973205566
Loss: 9.297639846801758
Loss: 5.751221656799316
Loss: 7.595761775970459
Loss: 7.319451808929443
Loss: 7.221586227416992
Loss: 8.557435989379883
Loss: 7.179542541503906
Loss: 6.736079216003418
Loss: 8.453967094421387
Loss: 8.435871124267578
Loss: 6.522689342498779
Loss: 6.860531806945801
Loss: 8.072932243347168
Loss: 6.386556625366211
Loss: 6.640663146972656
Loss: 6.838127136230469
Loss: 7.408633232116699
Loss: 7.719769477844238
Loss: 6.230581283569336
Loss: 5.880599498748779
Loss: 5.524681568145752
Loss: 

### Task 8

Like what we did in homework 2, before performing evaluation, we need to convert the label sequences to name entity brackets. A name entity is denoted as $E(i, j)$ where $E$ is the category, $(i,j)$ refer to the span that the entity covers ($i \leq j$).
Metrics are defined as:
- Precision = (# of correct entities in the predicted sequences)/(# of total entities in the predicted sequences)

- Recall = (# of correct entities in the predicted sequences)/(# of correct entities in the gold sequences)

- F1 score = 2\*precision\*recall/(precision+recall)

Let us see the example in the part of Dataset again.

**O** did

**B-DIRECTOR** sofia

**I-DIRECTOR** coppola

**O** direct

**O** any

**B-GENRE** adventure

**O** films

The gold name entities can be represented as: $DIRECTOR(1, 2)$, $GENRE(5, 5)$, assume the predicted name entities as $DIRECTOR(1, 2)$, $GENRE(5, 5)$, $TITLE(6, 6)$, then the precision=2/3, the recall=2/2, and the F1 score=0.8.


Let us work as following:
- Write a function to do the convertion.
- Write a function to compute the metrics given the predicted label sequences and gold label sequences.

In [ ]:
#Write your code here

In [27]:
def Span(a, b, c):
    return tuple((a, b, c))

def evaluate(real_tags, pred_tags):
    '''
    Evaluate based on the brackets including the label and the positions
    '''
    p = 0
    num = len(real_tags)
    total_entity = 0
    total_predict = 0

    for i in range(num):

        output = real_tags[i]
        prediction = pred_tags[i]
        #convert to span
        output_spans = set()
        start = -1
        previous = ' '
        for i in range(len(output)):
            if output[i].startswith("B-"):
                if previous.startswith("B-"):
                    output_spans.add(Span(i-1, i-1, previous[2:]))
                if previous.startswith("I-"):
                    output_spans.add(Span(start, i-1, previous[2:]))
                if i == (len(output)-1):#The last one begins with B
                    output_spans.add(Span(i, i, output[i][2:]))
                start = i
            if output[i].startswith("I-"):
                if i == (len(output)-1):#The last one begins with I
                    output_spans.add(Span(start, i, output[i][2:]))
            if output[i].startswith("O"):
                if previous.startswith("B-"):
                    output_spans.add(Span(i-1, i-1, previous[2:]))
                if previous.startswith("I-"):
                    output_spans.add(Span(start, i-1, previous[2:]))   
            previous = output[i]
        #print('Output spans:', output_spans)
        predict_spans = set()
        start = -1
        previous = ' '
        for i in range(len(prediction)):
            if prediction[i].startswith("B-"):
                if previous.startswith("B-"):
                    predict_spans.add(Span(i-1, i-1, previous[2:]))
                if previous.startswith("I-"):
                    predict_spans.add(Span(start, i-1, previous[2:]))
                if i == (len(output)-1):#The last one begins with B
                    predict_spans.add(Span(i, i, output[i][2:]))
                start = i
            if prediction[i].startswith("I-"):
                if i == (len(output)-1):#The last one begins with I
                    predict_spans.add(Span(start, i, prediction[i][2:]))
            if prediction[i].startswith("O"):
                if previous.startswith("B-"):
                    predict_spans.add(Span(i-1, i-1, previous[2:]))
                if previous.startswith("I-"):
                    predict_spans.add(Span(start, i-1, previous[2:]))   
            previous = prediction[i]
        
        #print('Predict spans:', predict_spans)

        total_entity += len(output_spans)
        total_predict += len(predict_spans)
        p += len(predict_spans.intersection(output_spans))
    
    print(p, total_entity, total_predict)
    precision = p * 1.0 / total_predict * 100 if total_predict != 0 else 0
    recall = p * 1.0 / total_entity * 100 if total_entity != 0 else 0
    fscore = 2.0 * precision * recall / (precision + recall) if precision != 0 or recall != 0 else 0

    return [precision, recall, fscore]

In [173]:
model.eval()
index = 0
pred_label_list = []
while index < len(dev_sent_words):
    batch_sents, batch_tags, seq_lens, index = data_generator(dev_sent_words, 
                                                              dev_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model.decoder(batch_sents, seq_lens)
    for label_seq in pred_labels:
        pred_labels = [id2label[t] for t in label_seq]
        pred_label_list.append(pred_labels)

### Task 9

Finally, we have built a sequence labeling system step by step. Let us explore the power of it on our NER task:
- Tune hyperparameters on the development set based on the F1 score using the pre-trained Glove word embeddings.
- Evaluate the performance on the test set, report the metrics.
- Choose 5 sentences in the test set randomly, display the predicted label sequences as well as the gold label sequences.

In [ ]:
#Write your code here

In [180]:
pred_label_list = []
index = 0
while index < len(test_sent_words):
    batch_sents, batch_tags, seq_lens, index = data_generator(test_sent_words, 
                                                              test_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model.decoder(batch_sents, seq_lens)
    for label_seq in pred_labels:
        pred_labels = [id2label[t] for t in label_seq]
        pred_label_list.append(pred_labels)

In [183]:
evaluate(pred_label_list, test_sent_tags)

3804 5517 5339


[71.24929762127739, 68.9505165851006, 70.0810611643331]

## Comparative experiment

### Task 10 
Our current model is biLSTM+CRF, you may wonder how much the CRF can contribute to the NER task. Let us make a comparative experiment:
- Design a simple biLSTM label sequencing system that can predict the labels directly based on the biLSTM outputs.
- Train your biLSTM system, tune hyper-parameters and evaluate it on the test set.
- Run both biLSTM+CRF and biLSTM models several times, and analyse whether biLSTM+CRF outperforms biLSTM based on the results or not.

In this case, we can view this sequence labeling problem as a classification problem, namely, to label each word with its hidden output from LSTM.

In [ ]:
#Write your code here

In [78]:
class SimpleSequenceLabeling(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, path):
        super(SimpleSequenceLabeling, self).__init__()
        self.crf = LinearChainCRF(label_size)
        self.bilstm = LSTMEncoder(vocab_size, embedding_dim, hidden_dim)
        self.hidden2label_space = nn.Linear(hidden_dim, label_size)
        self.softmax = nn.Softmax(dim=1)
        self.loss = nn.NLLLoss()
        self.load_pretrained_emb(path)
        
        
    def forward(self, sentences, seq_lens):
        '''
        Compute the loss
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
        Return:
            out: log softmax output, float tensor
        '''
        hiddens = self.bilstm(sentences, seq_lens)
        #batch_size*word_num*hidden_size
        outputs = []
        for i, l in enumerate(seq_lens):
            out = self.hidden2label_space(hiddens[i, :l])
            out = self.softmax(out)
            out = torch.log(out)
            outputs.append(out)
        return outputs
    
    def pred_seq(self, seqs, seq_lens):
        outputs = self.forward(seqs, seq_lens)
        preds = []
        for out in outputs:
            pred = out.argmax(1)
            preds.append(pred)
        return preds
    
    def get_seq_loss(self, seqs, seq_lens, tag_seqs):
        '''
        Compute the loss
        Args:
            seqs: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
            tag_seqs: batch_size*word_num, long tensor
        Return:
            loss: loss, float
        '''
        outputs = self.forward(seqs, seq_lens)
        losses = []
        for i, out in enumerate(outputs):
            loss = self.loss(out, tag_seqs[i, :seq_lens[i]])
            loss = loss * seq_lens[i]
            losses.append(loss)
        return torch.stack(losses).sum()/seq_lens.sum()

    
    def load_pretrained_emb(self, path):
        '''
        Load pre-trained word embeddings from the path
        Arg:
            path: the binary file of local Glove embeddings
        '''
        with open(path, 'rb') as f:
            vectors = pickle.load(f)
            print("Loaded from {} with shape {}".format(path, vectors.shape))
            assert vectors.shape == self.bilstm.word_embeds.weight.size()
            self.bilstm.word_embeds.weight.data.copy_(torch.from_numpy(vectors))
            self.bilstm.word_embeds.weight.requires_grad = False
            print('embeddings loaded')

In [79]:
model = SimpleSequenceLabeling(len(vocab), 100, 64, len(label2id), 'data/vocab/local_emb.pkl')
optimizer = optim.Adam(model.parameters(), lr=0.001)

Loaded from data/vocab/local_emb.pkl with shape (7482, 100)
embeddings loaded


In [83]:
from time import time
epoch = 3
loop_num = int(len(train_sent_words)/32)
loss_func = nn.NLLLoss()
for i in range(epoch):
    start = time()
    model.train()
    for j in range(loop_num):
        optimizer.zero_grad()
        batch_sents, batch_tags, seq_lens = data_generator(train_sent_words, train_sent_tags)
        loss = model.get_seq_loss(batch_sents, seq_lens, batch_tags)
        loss.backward()
        optimizer.step()
        if j % 50 == 0:
            print('Loss:', loss.item())

Loss: 0.4073823392391205
Loss: 0.4119170606136322
Loss: 0.5569366812705994
Loss: 0.5293269157409668


KeyboardInterrupt: 

In [84]:
pred_label_list = []
index = 0
while index < len(test_sent_words):
    batch_sents, batch_tags, seq_lens, index = data_generator(test_sent_words, 
                                                              test_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model.pred_seq(batch_sents, seq_lens)
    for label_seq in pred_labels:
        pred_labels = [id2label[t.item()] for t in label_seq]
        pred_label_list.append(pred_labels)

In [85]:
evaluate(pred_label_list, test_sent_tags)

3552 5421 5339


[66.52931260535681, 65.5229662423907, 66.02230483271374]

## Design Challenge (optional)

- Now, think of a better design for developing improved sequence labeling systems using any model you like.
- Implement your model and run experiments on the NER datasets.
- Write a report to elaborate your model, analyse the performance.

In [ ]:
#Write your code here

### Free GPU Resources
We suggest you run neural models on machines with GPU(s). Google provides a free online platform [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb), a research tool for machine learning education and research. It’s a Jupyter notebook environment that requires no setup to use as common packages have been  pre-installed. Google users can have access to a Tesla T4 GPU (approximately 15G memory). Note that when you connect to a GPU-based VM runtime, you are given a maximum of 12 hours at a time on the VM.

It is convenient to upload local Jupyter Notebook files and data to Colab, please refer to the [tutorial](https://colab.research.google.com/notebooks/io.ipynb). 

Microsoft also provides online platform [Azure Notebooks](https://notebooks.azure.com/help/introduction) for data science and machine learning, there are free versions as well as free trials for new users with credits.